# Second-Level GLM Analysis and Multiple Comparisons

In the previous lesson we ran a first-level analysis on our data set. Each subject's activity was modeled using GLMs, and we saved those models to `first_level_models.p`. The next step is to combine these models to examine if the within-subject effects are consistent across the group, and thus significant at the group-level.

This lesson will begin by performing a second-level analysis of a contrast. We'll generate a 'publication-ready' figure and extract the information we need for a paper. After, we'll discuss a few different ways to adjust for multiple comparisons. 

In this lesson, we'll continue to use nilearn to analyze and visualize our data. Let's begin by importing what we need:
- Pickle to load in our models
- nibabel to load in a brain mask
- matplotlib to help with visualization
- The `SecondLevelModel` class from nilearn
- A few nilearn classes/functions that help with reporting our results
- Two nilearn functions that let us plot our stat maps on brains

In [ ]:
import pickle
import matplotlib.pyplot as plt
import nibabel as nib
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.reporting import get_clusters_table
from nilearn.plotting import view_img, plot_stat_map, plot_contrast_matrix

%matplotlib inline

In [3]:
with open('first_level_models.p', 'rb') as f:
    glms = pickle.load(f)

## 1. Second Level Analysis

### 1.1. Setting up our model

Second-level analysis is easily done with nilearn's `SecondLevelModel` class [see online documentation](https://nilearn.github.io/stable/modules/generated/nilearn.glm.second_level.SecondLevelModel.html#nilearn.glm.second_level.SecondLevelModel). Like with `FirstLevelModel`, we define it first, and then fit our second-level model. The first-level models we ran and saved last lesson will serve as inputs when fitting the second-level model.  

There are actually a few different ways we in which we can input our first-level results to `SecondLevelModel`, but this one is the most straightforward and thus recommended. More on this later. 

In [ ]:
second_level = SecondLevelModel()
second_level.fit(glms)

You'll notice that this took no time at all. This is because all of the heavy lifting was done in the first-level analysis. We actually didn't do any analysis here; we simply combined all of our data together. We can take a look at the design matrix, which will give us just the intercept:

In [ ]:
second_level.design_matrix_

We only get the intercept because our analysis is completely within-subjects. Each row in the design matrix above corresponds to a subject. If we had two different groups, for example, then another column would be added to distinguish these groups (e.g., left- vs right-handed). 

In other words, we will be comparing conditions within subjects, but not between them. Our second-level model simply pools all our subjects into one group, and then we can see if the model results for each subject in this group are consistent.

### 1.2 Defining a contrast

Essentially we want to set up a contrast that answers the following questions: Does a task consistently activate the same voxels in our sample? Does one condition consistently yield greater activity in the same voxels compared to another condition? 

This is all done by the `compute_contrast` method of `SecondLevelModel`. Here, we define the first-level effect/contrast of interest that will be performed on each subject. For instance, by choosing 'Finger' we ask: What areas in the brain are consistenly activated when subjects move their fingers?

This contrast will be run on each subject to produce their statistical maps (commonly called stat maps or parameter maps). Then, these stat maps are pooled together and one-sample t-tests are performed on each voxel. A final stat map of _t_ values is produced; each voxel in the brain has a corresponding _t_ value that tells us the extent to which voxels are greater or less than 0. Often, these _t_ values are converted to _z_ values.

We could have done this last lesson by running contrasts for each subject (like we did in our in-depth walkthrough of one subject). Then, we could analyze the stat maps of these contrasts directly. I recommend trying this out eventually, but nilearn can do all of this for us automatically.

All we need to do is provide the first-level contrast that we wish to use, the second-level stat type (default is the t-test, specified by `second_level_stat_type`), and what we want our final stat map to be (specified by `output_type`). By default, nilearn convert this final stat map to _z_-values and outputs a _z_ map. 

In [ ]:
z_map = second_level.compute_contrast(first_level_contrast='Arm', 
                                      second_level_stat_type='t', 
                                      output_type='z_score')

It's always good to check to make sure you entered in the correct contrast. A conventional way to visualize the contrast is to plot the weights of the contrast. We can do this by the `plot_contrast_matrix` function in the `reporting` module. 

Because we're using first-level contrasts, we can just take one of our first-level models and plug in its design matrix:

In [ ]:
fig, ax = plt.subplots()
plot_contrast_matrix('Arm', glms[0].design_matrices_[0], ax=ax)

Our contrast is extremely simple, so this seems silly. Arm is the only positive condition. This becomes more useful for more complex contrasts. Next, we'll visualize our stat map on the brain. We won't apply any thresholding and we'll visualize it interactively:

In [ ]:
view_img(z_map, resampling_interpolation='nearest')

Take time to interactively explore the results. What do you see? Are the results consistent with your expectations (e.g., a motor task activates motor cortex)? _Always take time for sanity checks_. It's important to explore the raw results before looking at significance.

### 1.3 Thresholding and presenting the data

Next, let's create a static 'publication ready' figure that nicely shows some of the biggest clusters. This requires two steps:

1. Thresholding the image:
    - Threshold the image appropriately. We'll threshold the image such that we only show significantly voxels using $\alpha$ = .01 (two-sided _p_ < .01; _z_ > 2.58)
    - Further threshold the image by only showing significant voxels that are part of a cluster. We'll define a cluster using the minimum voxel criteria _k_ > 10. This will remove small, likely spurious clusters of significant voxels, as well as random singletons.

2. Visualizing the thresholded image:
    - Display the image orthographically (all three perspectives are shown)
    - Select views that show our most prominent clusters
    - Make it aesthetically pleasing 

We'll use nilearn' `threshold_stats_img` function to complete 1), and then nilearn's `plot_stat_map` to complete 2).

In [ ]:
thresh_img, thresh = threshold_stats_img(z_map, alpha=.01, cluster_threshold=10)

# in addition to the image, it returns the computed z threshold
print('threshold:', thresh)

plot_stat_map(thresh_img, cut_coords=[24, 0, 55], draw_cross=False,
              display_mode='tiled')

In addition to showing our results, we need to give information about the significant clusters we obtained. We want to report cluster location, size, and the magnitude of it's activity. We can get all of this cluster information from the `get_clusters_table` function in the `reporting` module.

This function will return a pandas DataFrame with all of the information we need to report in a paper (other than the anatomical location):

In [ ]:
results = get_clusters_table(z_map, stat_threshold=thresh, cluster_threshold=10)
results

Congrats, you've just completed your first full GLM analysis! We can now say that our contrast consistently produces the above clusters in our sample. In other words, these regions are thought to be activated by arm movements. 

We applied a liberal threshold of $\alpha$ = 0.01 (or _z_=2.58), and _k_ > 10. Using a more stringent alpha than $\alpha$ = 0.05 protects us a bit from type 1 error. Remember, we performed thousands of t-tests (one for each voxel in the brain), and we need to correct for multiple-comparisons. There are more rigorous approaches to this, and this is a fairly huge topic in the literature.

## 2. Multiple Comparisons

### 2.1 Increasing significance or cluster thresholds

The simplest thing we can do is just increase our current thresholds. For instance, increasing $\alpha$ from .01 to .001. Or, we could increase our minimum cluster threshold, _k_. Test some of these options below by change the `alpha` and the `cluster_threshold` parameters: 

In [ ]:
thresh_img, thresh = threshold_stats_img(z_map, alpha=.001, cluster_threshold=10)
view_img(thresh_img, resampling_interpolation='nearest')

While informative as an excercise, we can see that playing around with thresholds is totally arbitrary. As well, trying out different combinations of thresholds is p-hacking in it's purest form. 

For a long time, the field stuck to arbitrary thresholds such as $\alpha$ = .01 and _k_ = 10. You might still see some papers that apply these types of corrections. I would be pretty wary of the smaller clusters in their results (or all clusters if they aren't massive, for that matter). 

Next, we'll move on to actual ways to correct for multiple comparisons.

### 2.2 Family-wise error rate correction

We can correct the family-wise error rate by using a [Bonferroni correction](https://en.wikipedia.org/wiki/Bonferroni_correction). From your basic stats class, remember that Bonferroni corrections adjust $\alpha$ base on the number of tests performed, such that:

$$ \text{significance threshold} = \frac{\alpha}{m} $$ 

Where $m$ is the number of tests performed.

For this we'll want to also use a brain mask because we only performed t-tests within brain voxels.

In [ ]:
brain_mask = nib.load('MNI_brain_mask.nii.gz')

thresh_img, thresh = threshold_stats_img(z_map, alpha=.05, height_control='bonferroni', 
                                   mask_img=brain_mask)
view_img(thresh_img, resampling_interpolation='nearest')

Yikes, we only get one significant voxel, despite the pretty large clusters we were seeing earlier. This is because we're correcting over _thousands_ of tests. If we even just performed 1000 tests, our threshold would be .05 / 1000 = .00005. You can try increasing `alpha` and see if any significant voxels do eventually show up.   

As you can see, Bonferroni corrections are overly conservative; they are seldom used.  

### 2.3 False-positive rate correction

A more liberal approach than the Bonferonni correction is the [False-Discovery Rate correction](https://en.wikipedia.org/wiki/False_discovery_rate). This will only adjust within significant voxels ('discoveries') such that only an expected proportion (typically _q_ = .05) of significant voxels are retained. 

In [ ]:
thresh_img, thresh = threshold_stats_img(z_map, alpha=.05, height_control='fdr', mask_img=brain_mask)
view_img(thresh_img, resampling_interpolation='nearest')

We can see that these results are much more sensible. However, our effects are _really_ strong! FDR-correction is sometimes used, but for more subtle effects, the field tends to use cluster-level correction.

### 2.4 Cluster-level Correction

Cluster-level correction is by far the most popular method to use to correct for multiple-comparisons. This method first thresholds the voxels based on some $\alpha$ value (the height threshold or cluster-forming threshold, typically .01 or .001). Then, only clusters greater than a certain size are retained (the cluster-extent threshold, typically p<0.05). We sort of did this earlier by setting the number of voxels in a cluster. However, this approach determines the minimum cluster size by computing a null distribution of cluster sizes and finding the cluster size associated with the cluster-extend threshold. 

The benefit here is that the minimum cluster size is empirically determined, often yielding more stringent thresholds than the arbitrary _k_ = 10. And, unlike multiple-comparison procedures discussed above, it takes into account the spatial information in our data--it is sensible to assume that a task or behaviour elicits activity in a cluster of voxels rather than one voxel in isolation. As such, we're ultimately interested in regions, rather than individual voxels. 

Unfortunately, nilearn doesn't have a good function for conventional cluster-level correction yet. It sort of does, but the details and the implementation are outside of the scope of this course. Besides, typically people use software such as AFNI, FSL, or SPM. 

For the purpose of the final project, we'll simply just accept results using $\alpha$ = .001, and some reasonable _k_ value (e.g., 15, 20), or FDR-corrected results. For your own research, I would highly recommend diving into the cluster-correction literature and deciding what the best approach is for you.

## 3. Excercise

Go back and test out several different contrasts of your choice. Check to see if the results align with what you had in mind. 